In [186]:
import os
from xml.etree import ElementTree as ET
from lxml import etree
import re
from loguru import logger
from zhon.hanzi import punctuation
import string
import cn2an
import pandas as pd

import json
from xml.sax.saxutils import unescape
import jieba
import jieba.analyse as analyse

import time
from gensim.models import KeyedVectors
import numpy as np

In [10]:
t0 = time.time()
word2vec = KeyedVectors.load('/root/pretrain-models/word2vec/Tencent_AILab_ChineseEmbedding.bin', mmap='r')
print(time.time() - t0)

6.660168409347534


In [14]:
title_exclude_list = [
    "交接",
    "串联",
    "新串联",
    "稍后",
    "提要",
    "开始语",
    "结束语",
    "（连放",
    "【连放"
]


def check_file(file):
    if os.path.exists(file):
        os.remove(file)
    fp = open(file, "w", encoding="utf-8")
    fp.close()


def parse_html(txt):
    # 解析txt
    xml_str = unescape(txt)  # html -> html str
    html = etree.HTML(xml_str)
    result = html.xpath("//p//text()")

    # 去掉[正文]末尾的空格
    result = [res.strip() for res in result]

    # 提取正文和导语之后的内容
    text = result[result.index("[正文]"):result.index("[导语]")]
    introduction = result[result.index("[导语]"):result.index("[编后]")]
    mark_txt = ""
    for introduction_str in introduction:
        if len(introduction_str) > 10:
            mark_txt = introduction_str
            break
    if mark_txt == "":
        for text_str in text:
            if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("素材"):
                mark_txt = text_str
                break
    last_str = ""
    for text_str in text[::-1]:
        if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("东方卫视"):
            last_str += text_str
#             logger.info(last_str)
            break
    if last_str == "":
        for introduction_str in introduction[::-1]:
            if len(introduction_str) > 10:
                last_str = introduction_str
                break
    txt_str = ""
    for text_enum in text:
           txt_str = txt_str + text_enum
    intro_str = ""
    for intro_enum in introduction:
           intro_str = intro_str + intro_enum

    for i in punctuation:
        txt_str = txt_str.replace(i, "")
    for i in string.punctuation:
        txt_str = txt_str.replace(i, "")
    txt_str = txt_str.replace(" ", "")
    txt_str = cn2an.transform(txt_str, "an2cn")

    for i in punctuation:
        intro_str = intro_str.replace(i, "")
    for i in string.punctuation:
        intro_str = intro_str.replace(i, "")
    intro_str = intro_str.replace(" ", "")
    intro_str = cn2an.transform(intro_str, "an2cn")
    return txt_str, intro_str, mark_txt, last_str


def analysis_series_bill(xml_path):
    # 解析xml
    root = ET.parse(xml_path).getroot()
    # 遍历txt
    data = []
    for node in root.iter("text"):
        # num
        num = node.find("num").text
        title = node.find("title").text
        txt = node.find("txt").text
        words = node.find("words").text

        # 解析txt
        text, introduction, mark_txt, last_txt = parse_html(txt)

        audiotime = node.find("audiotime").text
        runtime = node.find("runtime").text
        totaltime = node.find("totaltime").text

        # for exclude_title in title_exclude_list:
        #     if re.match(exclude_title, title, flags=0):
        #         mark_txt = ""

        line = {"num": num, "title": title, "txt": text, "introduction": introduction, "audio_time": audiotime,
                "runtime": runtime, "total_time": totaltime, "mark_txt": mark_txt, "last_txt": last_txt, "words": words}
        data.append(line)

    return data

In [175]:
data4df = {
    'text': [],
    'pair': [],
    'label': []
}

In [187]:
df = pd.DataFrame(data4df)
df

,text,pair,label
0,导语提要全国新冠疫苗接种累计已超过十亿剂次接种剂次和覆盖人群数量均居全球首位我国页岩油勘探获...,导语正文,0
1,导语正文,导语今天是二零二一年六月二十日星期日欢迎收看东方新闻我是小雪在上海的演播室向各位问好柏我是柏...,0
2,导语今天是二零二一年六月二十日星期日欢迎收看东方新闻我是小雪在上海的演播室向各位问好柏我是柏...,导语新华社昨天播发长篇通讯青史如鉴耀千秋以习近平同志为核心的党中央领导中国共产党历史展览馆建...,0
3,导语新华社昨天播发长篇通讯青史如鉴耀千秋以习近平同志为核心的党中央领导中国共产党历史展览馆建...,导语在庆祝中国共产党成立一百周年之际中宣部新命名了一百一十一个全国爱国主义教育示范基地现配今...,0
4,导语在庆祝中国共产党成立一百周年之际中宣部新命名了一百一十一个全国爱国主义教育示范基地现配今...,导语今天台湾陆委会对香港特区政府暂停香港驻台机构妄言攻击对此国台办发言人马晓光应询表示中国香...,0
...,...,...,...
264,正文一百八十四票赞成二票反对三票弃权这是自一九九二年以来联合国大会连续近三零年以压倒性多数通...,正文据伊朗国家电视台报道伊朗安全部门昨天挫败一起VTR针对伊朗原子能组织建筑的破坏行动破坏者...,0
265,正文据伊朗国家电视台报道伊朗安全部门昨天挫败一起VTR针对伊朗原子能组织建筑的破坏行动破坏者...,正文软件截图十多年前迈克菲曾是众多计算机用户装机时的必备杀毒软件其创始人约翰迈克菲却在公司鼎...,0
266,正文软件截图十多年前迈克菲曾是众多计算机用户装机时的必备杀毒软件其创始人约翰迈克菲却在公司鼎...,导语今天我们给大家的带来的新闻就是这些想要回看今天的东方新闻了解更多资讯请扫描二维码下载看看...,0
267,导语今天我们给大家的带来的新闻就是这些想要回看今天的东方新闻了解更多资讯请扫描二维码下载看看...,导语正文,0


In [190]:
for idx, row in df.iterrows():
    print(row['pair'])

导语正文
导语今天是二零二一年六月二十日星期日欢迎收看东方新闻我是小雪在上海的演播室向各位问好柏我是柏栩栩我们在上海的演播室向各位问好在快速浏览了今天的主要新闻之后我们来看详细报道正文
导语新华社昨天播发长篇通讯青史如鉴耀千秋以习近平同志为核心的党中央领导中国共产党历史展览馆建设纪实文章指出党的十九大闭幕后不久以习近平同志为核心的党中央作出重大决策建设中国共产党历史展览馆二零一八年九月十号工程正式开工展览馆的建设进展情况始终牵动着总书记的心去年九月二十九号习近平专门听取了工程建设和展览情况工作汇报强调要把不忘初心牢记使命作为一条红线把展览馆建成一个神圣殿堂成为共产党员受教育受洗礼的精神家园成为引导人民群众听党话跟党走的教育基地成为党史研究和宣传的权威阵地今年二月八号习近平又主持召开会议审议党史展览方案强调要通过有代表性说服力的展览元素讲清楚中国共产党诞生的历史背景中国人民选择中国共产党领导的历史必然性讲清楚党的守正创新与时俱进今年五月五号历经近一千天建设中国共产党历史展览馆正式竣工落成正文主副标题一起出
导语在庆祝中国共产党成立一百周年之际中宣部新命名了一百一十一个全国爱国主义教育示范基地现配今年五月刚刚竣工落成的中国共产党历史展览馆便位列其中上海新命名三处分别为国歌展示馆上海浦东开发开放主题展馆和上海光源科学研究平台此外还有扩容类一处为中共一大纪念馆此次命名后全国爱国主义教育示范基地总数达到五百八十五个正文
导语今天台湾陆委会对香港特区政府暂停香港驻台机构妄言攻击对此国台办发言人马晓光应询表示中国香港特区和中国台湾地区进行交往当然要在一个中国原则的政治基础上进行台驻港机构当然必须遵守香港特区基本法和有关法律当然不能有违反一个中国原则的任何言行我们坚决支持香港特区政府按照基本法和中央政府关于处理香港涉台问题的基本原则和政策处理涉台事务坚决支持遏阻台独和反制民进党当局插手香港事务的正当行为正文
导语正文来关注新冠肺炎疫情昨天一天广东省没有新增本土确诊病例和本土无症状感染者详细内容请柏栩栩介绍
正文T一国家卫健委通报昨天全天内地没有本土新增确诊病例和本土无症状感染者T二十一今天广东省东莞市教育局发布工作指引要求各学校采取分级分类疫情防控其中东莞封闭封控区域内的学校和培训机构暂停线下教学T二十二根据疫情防控部署安排东莞图书馆鸦片战争博物馆等多个场所即日起暂停对外开放T三肇

In [184]:

# label_list = [[7,8],[8,9],[44,45],[45,46],[46,47],[47,48],[50,51]]
# label_list = [[7,8],[8,9],[11,12],[22,23]]
# label_list = [[12,13],[13,14],[23,24],[36,37],[37,38],[38,39],[39,40],[41,42],[42,43],[43,44]]
# label_list = [[10,11],[31,32],[32,33],[38,39],[39,40]]
label_list = [[18,19],[20,21],[22,23],[29,30],[31,32],[32,33],[40,41],[41,42]]

for idx in range(len(xml_20)-1):
    current_text = xml_20[idx]['txt'] if len(xml_20[idx]['txt']) > 200 else (xml_20[idx]['introduction'] + xml_20[idx]['txt'])
    next_text = xml_20[idx+1]['txt'] if len(xml_20[idx+1]['txt']) > 200 else (xml_20[idx+1]['introduction'] + xml_20[idx+1]['txt'])
    data4df['text'].append(current_text)
    data4df['pair'].append(next_text)
    if [idx+1, idx+2] in label_list:
        data4df['label'].append(1)
    else:
        data4df['label'].append(0)

703

In [183]:
xml_20 = analysis_series_bill('../data/news/20210624.xml')
xml_20

[{'num': '1',
  'title': '东方新闻提要',
  'txt': '正文头条标版不上标题习近平向一带一路亚太区域国际合作高级别会议发表书面致辞',
  'introduction': '导语提要习近平向一带一路亚太区域国际合作高级别会议发表书面致辞倡议建设更加紧密的一带一路伙伴关系共同推动构建人类命运共同体长三角地区纪检监察机关区域合作会议在沪举行三省一市签约建立协作机制李强指出发挥全面从严治党的政治保障作用为国家战略深入实施更好保驾护航东北华北十多条河流超警戒水位黑龙江干流或将出现六零年一遇洪水已启动Ⅰ级防汛应急响应苹果日报涉嫌违反香港国安法自主停止运作外交部驻港公署指出世界上没有凌驾法律之上的新闻自由商务部透露澳大利亚滥用贸易救济中国已在世贸组织起诉澳方相关反倾销和反补贴措施俄罗斯称英国军舰侵犯俄领海被实弹射击警告英方反驳说所谓开火其实是射击演习',
  'audio_time': '61',
  'runtime': '30',
  'total_time': '91',
  'mark_txt': '习近平向"一带一路"亚太区域国际合作高级别会议发表书面致辞，倡议建设更加紧密的"一带一路"伙伴关系，共同推动构建人类命运共同体。##',
  'last_txt': '亚太区域国际合作高级别会议',
  'words': '337'},
 {'num': '2',
  'title': '【已锁定】摇臂过场音乐',
  'txt': '正文',
  'introduction': '导语',
  'audio_time': '0',
  'runtime': '10',
  'total_time': '10',
  'mark_txt': '',
  'last_txt': '',
  'words': '0'},
 {'num': '3',
  'title': '开始语',
  'txt': '正文',
  'introduction': '导语今天是二零二一年六月二十四日星期四欢迎收看东方新闻我是李菡我是于飞我们在上海的演播室向各位问好在快速浏览了今天的主要新闻之后我们来看详细报道',
  'audio_time': '15',
  'runtime': '0',
  'total_time': '15',
  'mark_txt'

In [132]:
def keyword_extract(text, topK=3, remove_stopwords=True, method='tf-idf') -> dict:
    cus_stopwords = [
        '导语',
        '正文',
        '副标题',
        '一起',
        '标题',
        '处上'
    ]
    if remove_stopwords is True:
        for i in cus_stopwords:
            text = text.replace(i, '')
    if method == 'tf-idf':
        return analyse.extract_tags(text, topK=topK)
    return analyse.textrank(text, topK=topK)

In [95]:
keyword_extract('正文郭宪纲中国国际问题研究院研究员美国军舰经常在波斯湾游弋靠近伊朗水域很近伊朗为了维护自身的安全对美国的军舰实施抵近监控措施但这种监控措施不能演变为擦枪走火这就需要莱希对此加以管控伊拉克反美武装与伊朗关系密切是伊朗打击美国的一张王牌但对于莱希来讲这张牌用得过分也会给伊朗带来负面作用')

['伊朗', '莱希', '监控', '宪纲', '抵近']

In [154]:
res = []
for idx in range(len(xml_20)-1):
    current_keyword = keyword_extract(xml_20[idx]['introduction'] + xml_20[idx]['txt'], topK=5, method='tf-idf')
#     print(current_keyword)
    next_keyword = keyword_extract(xml_20[idx+1]['introduction'] + xml_20[idx + 1]['txt'], topK=5, method='tf-idf')
#     print(next_keyword)
    current_keyword_vec = keywords2vec(current_keyword)
    next_keyword_vec = keywords2vec(next_keyword)
    if sim_cosin(current_keyword_vec, next_keyword_vec) > 0.9:
        res.append((idx+1, idx+2))
#     print('num {} sim as num {} is {}'.format(idx+1, idx+2, sim_cosin(current_keyword_vec, next_keyword_vec)))
    
res

[(43, 44), (44, 45), (45, 46)]

In [102]:
def keywords2vec(lst):
    if len(lst) == 0:
        return np.array([0])
    try:
        res = word2vec[lst[0]].copy()
    except KeyError:
        sub_list = compute_ngrams(lst[0])
        try:
            res = word2vec[sub_list[0]].copy()
        except KeyError:
            res = np.zeros(200)
        for i in sub_list[1:]: # oor bug
            try:
                res += word2vec[i].copy()
            except KeyError:
                pass
    for i in lst[1:]:
        try:
            wcv = word2vec[i].copy()
        except KeyError:
            sub_list = compute_ngrams(lst[0])
            try:
                wcv = word2vec[sub_list[0]].copy()
            except Keyerror:
                wcv = np.zeros(200)
            for i in sub_list[1:]: # oor bug
                try:
                    wcv += word2vec[i].copy()
                except Keyerror:
                    pass
        res += wcv
    return res

In [98]:
np.array(word2vec['座']).size

200

In [87]:
def compute_ngrams(word, min_n=1, max_n=1):
    extended_word = word
    ngrams = []
    for ngram_length in range(min_n, min(len(extended_word), max_n) + 1):
        for i in range(0, len(extended_word) - ngram_length + 1):
            ngrams.append(extended_word[i:i + ngram_length])
    return list(set(ngrams))

In [110]:
def sim_cosin(v_o, v_t):
    return 0.5 + 0.5 * np.dot(v_o, v_t) / (np.linalg.norm(v_o) * np.linalg.norm(v_t)) if (np.linalg.norm(v_o) * np.linalg.norm(v_t)) != 0 else 0

In [70]:
sim_cosin(np.array(word2vec['接种']), np.array(word2vec['疫苗']))

0.9416744043438916

1.7320508075688772